# Calgary Omnimap (Better Name Pending)
## A unified map containing the features of the City of Calgary's numerous map apps

### Currently implemented:
* Roads
    * Traffic Cameras (see 511)
    * Incidents
    * Detours
    * Parking ban status (is the mobile API accurate? Find a more official source)
* 511 (provincial, not municipal, but still relevant)
    * Cameras (both AB and YYC, providing improved coverage on provincial highways)
* Bikeways and Pathways
    * Park 'n Bike locations
    * Downtown CPA bike parking
    * Bike paths (currently just using a bike tileset for OSM)
* Pets 
    * Off leash areas
* Proposed +15 map(s)
    * +15 map (with colour coding for enclosed/open!)

### Not yet implemented from city apps:
* Roads
    * Snowplows (is this even public data?)
    * "Bluetooth travel times" (what the heck does this even mean?)
* Bikeways and Pathways
    * Pathway closures and detours (does not seem to be on Open Data?)
* Calgary Transit (AKA Transit App) and HASTINFO
    * Real time bus arrivals
    * Bus locations
    * Routes
        * Route advisories
* 511
    * Road conditions
    * AB 
    * Events
        * Merge roadwork and closures into ```detour_features```
        * Merge accidentsAndIncidents into ```traffic_incident_features```
    * Conditions
    
### Misc TODO:
* Separate pin types for closures/full detours and regular delays (may require looking at keywords in the descriptions)
        
### Should/Could only be Implemented in a Rewritten Production Version (i.e. not a notebook)
* User location (kinda needed for good navigation)
* Location search (via Google Maps, Foursquare, or similar)
* Automatic polling of sources
* Navigation
    * Holistic navigation for walking, biking, transit, or any combination of the three
    * Option to prioritize warmer paths (e.g. taking the +15, even if it's a couple minutes slower)
    * Option to prioritize scenic routes (e.g. parks, rivers, off-leash areas)
    
### Interesting observations:
* The parking ban status is not up to date in the Roads app, and this can only be ascertained by looking at the raw json (the user has no idea if the notice is accurate)
* The Roads app has a text box to navigate somewhere, but this just fires an intent to launch the user's preferred navigation app (usually Google Maps). This offers no advantage over, you know, opening Maps yourself.
* The Roads app UI is broken on Android Nougat (no background on layer menu)


In [20]:
import pandas as pd
import numpy as np
import folium, requests, branca, json
from folium import plugins
import polyline

In [3]:
with open('tileset_keys.json') as f:
    tile_keys = json.load(f)

## Collect and Process Data

In [4]:
# traffic_cameras = pd.read_csv("https://data.calgary.ca/resource/35kd-jzrv.csv?$select=description,url,longitude,latitude") # City list of cameras
# traffic_cameras['popup'] =  '<h2>' + traffic_cameras['description'] + \
#                             '</h2> <br> <a href=\"' + traffic_cameras['url'] + \
#                             '\" target=\"_blank\"> <img src=\"' + traffic_cameras['url'] + '\" width=100%> </a>'
# traffic_cameras.head()

In [5]:
traffic_cameras = pd.read_json("https://511.alberta.ca/api/v2/get/cameras") # 511 Alberta cameras database includes both city and provincial cameras! Pretty handy.
traffic_cameras.columns = [s.lower() for s in traffic_cameras.columns]
traffic_cameras['id_prefix'] = [s.split('.')[0] for s in traffic_cameras['id']]
def set_or_value(x):
    if len(set(x)) > 1:
        return set(x)
    else:
        return x.iloc[0]
traffic_cameras = traffic_cameras.groupby('id_prefix').aggregate(set_or_value) # Aggregate any entries of the same camera ID prefix for multiple rows (usually 3 camera setups at interchanges). These entries consistently share the same lat/long.

camera_link_format = '<a href=\"{0}\" target=\"_blank\"> <img src=\"{0}\" width=100%> </a>' # HTML format string for an image that links to its source
def make_camera_image_list(cameras):
    if type(cameras) is str:
        return camera_link_format.format(cameras)
    else:
        return '<br>'.join([camera_link_format.format(x) for x in cameras])

camera_popups = []

for index, row in traffic_cameras.iterrows():
    popup = '<h2>' + row['name'] + '</h2>' + \
            make_camera_image_list(row['url']) + '<br>' + \
            str(row['description']) # TODO Better handling for sets
    
    camera_popups.append(popup)

traffic_cameras['popup'] = camera_popups

In [6]:
traffic_incidents = pd.read_csv("https://data.calgary.ca/resource/y5vq-u678.csv?$select=latitude,longitude,incident_info,description")
traffic_incidents.dropna(subset=['latitude', 'longitude'], inplace=True)
traffic_incidents['popup'] = '<h2>Traffic Incident</h2><h3>'+ traffic_incidents['incident_info']+'</h3>'+traffic_incidents['description']
traffic_incidents.head()

,description,incident_info,latitude,longitude,popup
0,"Northbound is reduced to one left lane, southb...",MacLeod Trail and 36 Avenue SW,51.021328,-114.060915,<h2>Traffic Incident</h2><h3> MacLeod Trail an...


In [7]:
detours = pd.read_csv("https://data.calgary.ca/resource/q5fe-imxj.csv?$select=latitude,longitude,construction_info,description,start_dt,end_dt")
detours['popup'] = '<h2>Traffic Detour</h2><h3>'+detours['construction_info']+'</h3>'+detours['description']
detours.head()

,construction_info,description,end_dt,latitude,longitude,start_dt,popup
0,78 Avenue at 27 Street SE,6 AM February 28 to 5 PM March 29 <br>The road...,2019-03-29T17:00:00.000,50.983227,-113.995868,2019-02-28T06:00:00.000,<h2>Traffic Detour</h2><h3>78 Avenue at 27 Str...
1,10 Avenue at 9 Street SW,Daily 9 AM to 3 PM from March 4 to March 6 </b...,2019-03-06T15:00:00.000,51.043907,-114.083850,2019-03-04T09:00:00.000,<h2>Traffic Detour</h2><h3>10 Avenue at 9 Stre...
2,Country Hills Boulevard at 14 Street NW,9 AM March 4 to 3:30 PM March 8 <br>The right ...,2019-03-08T15:30:00.000,51.144100,-114.111894,2019-03-04T09:00:00.000,<h2>Traffic Detour</h2><h3>Country Hills Boule...
3,4 Street at 15 Avenue NE,12 PM to 10 PM March 23 <br>The road is closed...,2019-03-23T22:00:00.000,51.065989,-114.053252,2019-03-23T12:00:00.000,<h2>Traffic Detour</h2><h3>4 Street at 15 Aven...
4,9 Avenue at 9 Street SW,"Weekdays 9 AM to 3 PM, and 8 AM Saturday to 5 ...",2019-03-08T15:00:00.000,51.045182,-114.083766,2019-02-25T09:00:00.000,<h2>Traffic Detour</h2><h3>9 Avenue at 9 Stree...


In [8]:
park_and_bike = pd.read_csv("https://data.calgary.ca/resource/nc6z-cxzf.csv?$select=latitude,longitude,name,website,general_info")
park_and_bike['popup'] = '<h2>Park and Bike: '+ park_and_bike['name'] + '</h2>' + park_and_bike['general_info'] + '<br> <a href=\"' +\
                         park_and_bike['website'] + '\" target=\"_blank\">Website</a>'
park_and_bike.head()

,general_info,latitude,longitude,name,website,popup
0,Access the site from 50 Ave S.W. by driving do...,51.008715,-114.089999,Sandy Beach,http://www.calgary.ca/Transportation/TP/Pages/...,<h2>Park and Bike: Sandy Beach</h2>Access the ...
1,Access the site from highway 1 (16 Ave NW). Tw...,51.068723,-114.164822,Home Road,http://www.calgary.ca/Transportation/TP/Pages/...,<h2>Park and Bike: Home Road</h2>Access the si...
2,Parking lot is open but cyclists must cross th...,51.062069,-114.156115,Edworthy South,http://www.calgary.ca/Transportation/TP/Pages/...,<h2>Park and Bike: Edworthy South</h2>Parking ...
3,The west side of parking lot allows for best a...,51.063686,-114.150935,Edworthy North,http://www.calgary.ca/Transportation/TP/Pages/...,<h2>Park and Bike: Edworthy North</h2>The west...
4,The site can easily be accessed from Deerfoot ...,51.043349,-114.004929,Max Bell,http://www.calgary.ca/Transportation/TP/Pages/...,<h2>Park and Bike: Max Bell</h2>The site can e...


In [9]:
cpa_bike_parking = pd.read_csv("https://data.calgary.ca/resource/afcw-kkyc.csv?$select=latitude,longitude,name,web,indoor_stalls,outdoor_stalls")
cpa_bike_parking['popup'] = '<h2>CPA Bike Parking: ' + cpa_bike_parking['name'] + '</h2>' +\
                            'Indoor stalls: ' + cpa_bike_parking['indoor_stalls'].astype(str) + \
                            '<br>Outdoor stalls: ' + cpa_bike_parking['outdoor_stalls'].astype(str) + \
                            '<br> <a href=\"' + cpa_bike_parking['web'] + '\" target=\"_blank\">Website</a>'
cpa_bike_parking.head()

,indoor_stalls,latitude,longitude,name,outdoor_stalls,web,popup
0,44,51.048598,-114.063807,James Short Parkade,0,https://www.calgaryparking.com/findparking/bic...,<h2>CPA Bike Parking: James Short Parkade</h2>...
1,104,51.044497,-114.068188,City Centre Parkade,0,https://www.calgaryparking.com/findparking/bic...,<h2>CPA Bike Parking: City Centre Parkade</h2>...
2,16,51.049067,-114.076856,McDougall Parkade,0,https://www.calgaryparking.com/findparking/bic...,<h2>CPA Bike Parking: McDougall Parkade</h2>In...
3,12,51.050420,-114.074468,Lot 72,0,https://www.calgaryparking.com/findparking/bic...,<h2>CPA Bike Parking: Lot 72</h2>Indoor stalls...
4,27,51.045170,-114.074952,Centennial Parkade,22,https://www.calgaryparking.com/findparking/bic...,<h2>CPA Bike Parking: Centennial Parkade</h2>I...


In [10]:
plus15_shapes_json = json.loads(requests.get('https://data.calgary.ca/api/geospatial/kp44-4n8q?method=export&format=GeoJSON').text)
plus15_shapes = []
for feature in plus15_shapes_json['features']:
    if(feature['properties']['type'] is None):
        feature['style'] = {'fill':'gray', 'stroke':'black'}
    elif('Open to Sky' in feature['properties']['type']):
        feature['style'] = {'fill':'blue', 'stroke':'blue'}
    else:
        feature['style'] = {'fill':'red', 'stroke':'red'}
    plus15_shapes.append(feature)
plus15_shapes_json['features'][0]

{'type': 'Feature',
 'properties': {'feat_id': None,
  'revis_date': '2007-01-01',
  'access_hours': 'Monday - Friday, 7am - 9pm',
  'type': 'Open to Sky',
  'structure_type': 'Open to Sky',
  'modified_dt': None},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-114.069968732295, 51.048160705037],
     [-114.070043309403, 51.048171054294],
     [-114.070052693785, 51.048171182617],
     [-114.070036836522, 51.048402130165],
     [-114.07001202805, 51.048416222943],
     [-114.069979291572, 51.048415253678],
     [-114.069983145732, 51.048356997029],
     [-114.069962287258, 51.048342347268],
     [-114.069967656573, 51.048261733155],
     [-114.069991055576, 51.048248359509],
     [-114.069995232198, 51.048179810324],
     [-114.069968732295, 51.048160705037]]]]},
 'style': {'fill': 'blue', 'stroke': 'blue'}}

In [11]:
off_leash_json_res = requests.get('https://data.calgary.ca/api/geospatial/xrct-ap62?method=export&format=GeoJSON')
off_leash_json_res.encoding = 'utf-8' # Encoding is wonky with this dataset for some reason
off_leash_json = json.loads(off_leash_json_res.text, encoding = 'utf-8')
off_leash_shapes = []
for feature in off_leash_json['features']:
    if(feature['properties']['status'] == 'OPEN'):
        off_leash_shapes.append(feature)
off_leash_shapes[0]

{'type': 'Feature',
 'properties': {'agreement_info': None,
  'closed_dt': None,
  'fencing_info': None,
  'description': 'OGDEN',
  'maintained_by': 'CALGARY PARKS',
  'wam_parent_id': 'S-02003098',
  'steward': 'CALGARY PARKS',
  'asset_cd': 'OGD795',
  'parcel_location': '1951 69 AV SE',
  'status': 'OPEN',
  'maint_info': None,
  'off_leash_area_id': 'OGD-003',
  'category': 'COMMUNITY CLUSTER',
  'notes': None,
  'closed_reason': None,
  'opened_dt': '1988-12-08'},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-114.013045062001, 50.986508539205],
     [-114.013060713794, 50.986504896578],
     [-114.013070369812, 50.986502809859],
     [-114.013082560953, 50.986500399222],
     [-114.013095692199, 50.986498285143],
     [-114.013109080004, 50.986496845269],
     [-114.013112754567, 50.986496584152],
     [-114.01311303353, 50.986526223568],
     [-114.013112610218, 50.986793095499],
     [-114.013308392703, 50.986793208208],
     [-114.013471801145, 50.986793297357],


In [34]:
winter_roads = pd.read_json("https://511.alberta.ca/api/v2/get/winterroads")
winter_roads['polyline'] = winter_roads['EncodedPolyline'].apply(polyline.decode)
def statusToColor(condition):
    if (condition == 'Closed'):
        return 'red'
    elif ('Ptly Cvd' in condition):
        return 'yellow'
    elif ('Cvd' in condition):
        return 'orange'
    elif ('Bare Wet' in condition):
        return 'blue'
    elif ('Bare' in condition):
        return 'green'
    else:
        return 'black'
winter_roads['color'] = winter_roads['Primary Condition'].apply(statusToColor)
winter_roads['popup'] = winter_roads['LocationDescription'] + '<br>' + winter_roads['Primary Condition']
winter_roads.head()

,AreaName,EncodedPolyline,LocationDescription,Primary Condition,RoadwayName,Secondary Conditions,Visibility,polyline,color,popup
0,BANFF,{lhxHjqbdUi@e@{IcCeBkBa@k@a@UuDe@i@Bs@Pq@HcDw@...,Moraine Lake to Lake Louise Drive,Closed,Moraine Lake Rd,[],None,"[(51.33022, -116.18086), (51.33043, -116.18067...",red,Moraine Lake to Lake Louise Drive<br>Closed
1,WATERTON PARKS,w|gjHru`wT}k@mr@~`Adz@wBEuACgF_BkFmC|mAl[yB?uJ...,Cameron Lake to Little Prairie Gate,Closed,Akamina Parkway,[],None,"[(49.03388, -114.04138), (49.04107, -114.03315...",red,Cameron Lake to Little Prairie Gate<br>Closed
2,WATERTON PARKS,uiijHdb_wT}DaFiQaYgBcF{CwKYm@u@q@s@GqE^_AKo@s@...,Little Prairie Gate to Jct Hwy 5,Closed,Akamina Parkway,[],None,"[(49.04107, -114.03315), (49.04202, -114.03202...",red,Little Prairie Gate to Jct Hwy 5<br>Closed
3,WATERTON PARKS,}lpjHn`avTq@~@_@xBWn@a@n@[\a@Pm@Ic@Y]WYc@Ki@Ae...,Jct Hwy 5 to Red Rock Canyon,Closed,RedRock Parkway,[],None,"[(49.07743, -113.87928), (49.07768, -113.8796)...",red,Jct Hwy 5 to Red Rock Canyon<br>Closed
4,BANFF,kuiwHf}h`Uy@H{@Za@L_BlA_@H[Ak@Ma@Am@RaLvG??_@L...,St. Julien Way to Tunnel Mountain Rd,Closed,Tunnel Mountain Dr,[],None,"[(51.17286, -115.55812), (51.17315, -115.55817...",red,St. Julien Way to Tunnel Mountain Rd<br>Closed


In [13]:
parking_ban_status = json.loads(requests.get('http://cocnmp.com/snic/parking_ban_status.php?output=json').text)
parking_ban_status['message'] + ' as of ' + parking_ban_status['lastUpdate']

'Snow route parking bans are not in effect as of February 20 2019, 5:01 PM'

In [14]:
def add_df_to_featuregroup(in_df, in_featuregroup, color, icon, prefix='fa'): # Takes a dataframe with latitude, longitude, and an html popup field
    for index, row in in_df.iterrows():
        if (row['latitude'] is not np.NaN, row['longitude']):
            folium.Marker(
                location=[row['latitude'], row['longitude']],
                popup=row['popup'],
                icon=folium.Icon(prefix=prefix,icon=icon,color=color)
            ).add_to(in_featuregroup)

In [51]:
def embed_map(m, name):
    from IPython.display import IFrame

    m.save(name+'.html')
    map_file = open(name+'.html')
    injected_html = open('injected.html').read()
    map_html = map_file.read()
    map_file.close()
    map_html = map_html.replace('</head>', injected_html+'\n</head>')
    map_file = open(name+'.html', 'w')
    map_file.write(map_html)
    map_file.close()
    return IFrame(name+'.html', width='100%', height='750px')

In [55]:
# Draw final map! 
calgary_map = folium.Map(location=[51.0486, -114.0708], zoom_start=11, attr='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors')
roads_features = folium.FeatureGroup(name='Roads')
bike_features = folium.FeatureGroup(name='Cycling')
walking_features = folium.FeatureGroup(name='Walking')

# Roads
calgary_map.add_child(roads_features)

traffic_camera_features = plugins.FeatureGroupSubGroup(roads_features, '- Traffic Cameras', show=False)
add_df_to_featuregroup(traffic_cameras, traffic_camera_features, 'gray', 'camera')
calgary_map.add_child(traffic_camera_features)

traffic_incident_features = plugins.FeatureGroupSubGroup(roads_features, '- Traffic Incidents')
add_df_to_featuregroup(traffic_incidents, traffic_incident_features, 'red', 'car')
calgary_map.add_child(traffic_incident_features)

detour_features = plugins.FeatureGroupSubGroup(roads_features, '- Traffic Detours')
add_df_to_featuregroup(detours, detour_features, 'orange', 'exclamation-triangle')
calgary_map.add_child(detour_features)

winter_roads_features = plugins.FeatureGroupSubGroup(roads_features, '- Road Conditions')
for index, row in winter_roads.iterrows():
    thisLine = folium.PolyLine(row['polyline'], color=row['color'])
    folium.Popup(row['popup']).add_to(thisLine)
    thisLine.add_to(winter_roads_features)
calgary_map.add_child(winter_roads_features)

# Bike
calgary_map.add_child(bike_features)

bike_parking_features = plugins.FeatureGroupSubGroup(bike_features, '- Bike Parking')
add_df_to_featuregroup(park_and_bike, bike_parking_features, 'green', 'bicycle')
add_df_to_featuregroup(cpa_bike_parking, bike_parking_features, 'darkgreen', 'bicycle')
calgary_map.add_child(bike_parking_features)

# Walking
def str_or_unknown(s):
    return 'unknown' if s is None else str(s)
def str_or_blank(s):
    return '' if s is None else str(s)
calgary_map.add_child(walking_features)

plus_15_features = plugins.FeatureGroupSubGroup(walking_features, '- Plus 15')
for shape in plus15_shapes:
    feature = folium.GeoJson(shape, style_function=lambda x: {
                            'color' : x['style']['stroke'],
                            'weight' : 1,
                            'opacity': 0.75,
                            'fillColor' : x['style']['fill'],
                            })
    folium.Popup('<h2>Plus 15</h2>Hours: ' + str_or_unknown(shape['properties']['access_hours']) + '<br>Type: ' + str_or_unknown(shape['properties']['type'])).add_to(feature)
    plus_15_features.add_child(feature)
calgary_map.add_child(plus_15_features)

off_leash_features = plugins.FeatureGroupSubGroup(walking_features, '- Off-Leash Areas', show=False)
for shape in off_leash_shapes:
    feature = folium.GeoJson(shape, style_function=lambda x: {
                            'color' : 'green',
                            'weight' : 1,
                            'opacity': 0.75,
                            'fillColor' : 'green',
                            })
    folium.Popup('<h2>Off-Leash Area</h2><h3>' + str_or_unknown(shape['properties']['description']) \
                 + '<br>' + str_or_blank(shape['properties']['fencing_info'])).add_to(feature)
    off_leash_features.add_child(feature)
calgary_map.add_child(off_leash_features)

# Stick it all together
user_circle = folium.CircleMarker([51.046309, -114.068672])
folium.Popup('You are here').add_to(user_circle)
user_circle.add_to(calgary_map)

folium.TileLayer('CartoDB dark_matter', name='CartoDB Dark', attr='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>').add_to(calgary_map) # TODO Satellite layer
folium.TileLayer('https://{s}.tile.thunderforest.com/cycle/{z}/{x}/{y}.png?apikey='+tile_keys['thunderforest'],
                 name='Bike Paths',
                 attr='Tile Data &copy; <a href="http://www.thunderforest.com/">Thunderforest</a>, &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
).add_to(calgary_map)
folium.TileLayer('https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                 name='Satellite',
                 attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
).add_to(calgary_map)

folium.LayerControl().add_to(calgary_map)

embed_map(calgary_map, 'Omnimap')

In [17]:
calgary_map.save('Omnimap.html')